In [6]:
%tensorflow_version 2.x

UsageError: Line magic function `%tensorflow_version` not found.


In [2]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install -U scikit-image

     |████████████████████████████████| 30.2 MB 32 kB/s  eta 0:00:01
  Attempting uninstall: scikit-image
    Found existing installation: scikit-image 0.18.1
    Uninstalling scikit-image-0.18.1:
      Successfully uninstalled scikit-image-0.18.1
Note: you may need to restart the kernel to use updated packages.


In [13]:
pip install pydot

Note: you may need to restart the kernel to use updated packages.


In [15]:
pip install graphviz

Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import skimage.io as io
import numpy as np

import tensorflow as tf

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12705347438102860883
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 2911292622
locality {
  bus_id: 1
  links {
  }
}
incarnation: 635298342322149553
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1050 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [3]:
tf.debugging.set_log_device_placement(True)

# Create some tensors
a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
c = tf.matmul(a, b)

Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op MatMul in device /job:localhost/replica:0/task:0/device:GPU:0


In [2]:
!cd /media/viktor/USB DISK

/bin/bash: line 0: cd: too many arguments


In [3]:
!mkdir -p data

In [4]:
if 1:
    !mkdir -p data

    # !cd data && wget http://images.cocodataset.org/zips/train2017.zip 
    # !cd data && wget http://images.cocodataset.org/zips/val2017.zip 
    !cd data && wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip 

    # !cd data && unzip -q train2017.zip
    # !cd data && unzip -q val2017.zip
    !cd data && unzip -q annotations_trainval2017.zip

    !cd data && git clone https://github.com/cocodataset/cocoapi
    !cd data/cocoapi/PythonAPI && make

--2021-10-14 19:47:04--  http://images.cocodataset.org/annotations/annotations_trainval2017.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.217.39.220
Connecting to images.cocodataset.org (images.cocodataset.org)|52.217.39.220|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 252907541 (241M) [application/zip]
Saving to: ‘annotations_trainval2017.zip’

annotations_trainva 100%[===================>] 241,19M   283KB/s    in 7m 28s  

2021-10-14 19:54:33 (551 KB/s) - ‘annotations_trainval2017.zip’ saved [252907541/252907541]

Cloning into 'cocoapi'...
remote: Enumerating objects: 975, done.
remote: Total 975 (delta 0), reused 0 (delta 0), pack-reused 975
Receiving objects: 100% (975/975), 11.72 MiB | 1.10 MiB/s, done.
Resolving deltas: 100% (576/576), done.
python setup.py build_ext --inplace
running build_ext
cythoning pycocotools/_mask.pyx to pycocotools/_mask.c
/home/viktor/anaconda3/lib/python3.8/site-packages/Cython/Compiler/Main.py:369: Fut

In [4]:
COCO_ROOT = './data/'
import sys
sys.path.insert(0, os.path.join(COCO_ROOT, 'cocoapi/PythonAPI'))
from pycocotools.coco import COCO

ModuleNotFoundError: No module named 'pycocotools._mask'

In [9]:
class Dataset():

    def crop_images(self, img, inp_size, random_crop=False):
        shape = tf.shape(img)
        pad = (
            [0, tf.maximum(inp_size - shape[0], 0)],
            [0, tf.maximum(inp_size - shape[1], 0)],
            [0, 0],
        )
        img = tf.pad(img, pad)

        if random_crop:
            img = tf.image.random_crop(img, (inp_size, inp_size, shape[2]))
        else: # central crop
            shape = tf.shape(img)
            ho = (shape[0] - inp_size) // 2
            wo = (shape[1] - inp_size) // 2
            img = img[ho:ho+inp_size, wo:wo+inp_size, :]

        return img

    def train_dataset(self, batch_size, epochs, inp_size):

        def item_to_images(item):
            random_crop = True
            img_combined = tf.py_function(self.read_images, [item], tf.uint8)
            img_combined = self.crop_images(img_combined, inp_size, random_crop)

            img = tf.cast(img_combined[...,:3], tf.float32) / np.float32(255.)
            mask_class = tf.cast(img_combined[...,3:4], tf.float32)
            return img, mask_class

        dataset = tf.data.Dataset.from_tensor_slices(self.img_list)
        dataset = dataset.shuffle(buffer_size=len(self.img_list))
        dataset = dataset.map(item_to_images)
        dataset = dataset.repeat(epochs)
        dataset = dataset.batch(batch_size, drop_remainder=True)

        return dataset

    def val_dataset(self, batch_size, inp_size):

        def item_to_images(item):
            random_crop = False
            img_combined = tf.py_function(self.read_images, [item], tf.uint8)
            img_combined = self.crop_images(img_combined, inp_size, random_crop)

            img = tf.cast(img_combined[...,:3], tf.float32) / np.float32(255.)
            mask_class = tf.cast(img_combined[...,3:4], tf.float32)
            return img, mask_class

        dataset = tf.data.Dataset.from_tensor_slices(self.img_list)
        dataset = dataset.map(item_to_images)
        dataset = dataset.batch(batch_size, drop_remainder=True)

        return dataset

In [10]:
class COCO_Dataset(Dataset):

    def __init__(self, sublist):
        ann_file_fpath = os.path.join('/media/viktor/Дата/data/annotations', 'instances_'+sublist+'2017.json')
        self.coco = COCO(ann_file_fpath)
        self.cat_ids = self.coco.getCatIds(catNms=['person'])
        self.img_list = self.coco.getImgIds(catIds=self.cat_ids)

    def read_images(self, img_id):
        img_id = int(img_id.numpy())
        img_data = self.coco.loadImgs(img_id)[0]
        img_fname = '/'.join(img_data['coco_url'].split('/')[-2:])

        img = io.imread(os.path.join('/media/viktor/Дата/data', img_fname))
        if len(img.shape) == 2:
            img = np.tile(img[..., None], (1, 1, 3))

        ann_ids = self.coco.getAnnIds(imgIds=img_data['id'], catIds=self.cat_ids, iscrowd=None)
        anns = self.coco.loadAnns(ann_ids)
        mask_class = np.zeros((img.shape[0], img.shape[1]), dtype=np.uint8)
        for i in range(len(anns)):
            mask_class += self.coco.annToMask(anns[i])
        mask_class = (mask_class > 0).astype(np.uint8)

        img_combined = np.concatenate([img, mask_class[..., None]], axis=2)

        return img_combined

In [12]:
COCO_dataset_train = COCO_Dataset('train')
COCO_dataset_val = COCO_Dataset('val')

loading annotations into memory...
Done (t=11.55s)
creating index...
index created!
loading annotations into memory...
Done (t=0.37s)
creating index...
index created!


In [13]:
class ASPPBlock(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.conv1 = tf.keras.layers.Conv2D(256, (1, 1), padding='same', activation='relu')
        self.conv2 = tf.keras.layers.Conv2D(256, (3, 3), dilation_rate=6, padding='same', activation='relu')
        self.conv3 = tf.keras.layers.Conv2D(256, (3, 3), dilation_rate=12, padding='same', activation='relu')
        self.conv4 = tf.keras.layers.Conv2D(256, (3, 3), dilation_rate=18, padding='same', activation='relu')
        self.conv5 = tf.keras.layers.Conv2D(256, (1, 1), padding='same', activation='relu')

    def call(self, inp, is_training=False):
        out1 = self.conv1(inp)
        out2 = self.conv2(inp)
        out3 = self.conv3(inp)
        out4 = self.conv4(inp)
        out = tf.concat([out1, out2, out3, out4], axis=3)
        out = self.conv5(out)
        return out

In [14]:
class CCM(tf.keras.layers.Layer):
    def __init__(self, filters, kernel_size, **kwargs):
        super().__init__()
        self.conv1 = tf.keras.layers.Conv2D(filters, kernel_size, **kwargs)
        self.conv2 = tf.keras.layers.Conv2D(filters, kernel_size, **kwargs)
        self.maxpool = tf.keras.layers.MaxPool2D((2, 2))

    def call(self, inp, training=False):
        out = self.conv1(inp)
        out2 = self.conv2(out)
        out1 = self.maxpool(out2)
        return out1, out2

In [15]:
class CTConCC(tf.keras.layers.Layer):
    def __init__(self, filters, kernel_size, strides, **kwargs):
        super().__init__()
        self.conv_trans = tf.keras.layers.Conv2DTranspose(filters, kernel_size, strides=strides, **kwargs)
        self.conv1 = tf.keras.layers.Conv2D(filters, kernel_size, **kwargs)
        self.conv2 = tf.keras.layers.Conv2D(filters, kernel_size, **kwargs)

    def call(self, inp, out2, training=False):
        out = self.conv_trans(inp)
        out = tf.concat([out2, out], axis=3)
        out = self.conv1(out)
        out = self.conv2(out)
        return out

In [16]:
def build_model():
    x = tf.keras.layers.Input((128, 128, 3))
    out, out1 = CCM(64,  (3, 3), padding='same', activation='relu')(x)    
    out, out2 = CCM(128, (3, 3), padding='same', activation='relu')(out)
    out, out3 = CCM(256, (3, 3), padding='same', activation='relu')(out)
    out, out4 = CCM(512, (3, 3), padding='same', activation='relu')(out)

    out = tf.keras.layers.Conv2D(1024, (3, 3), padding='same', activation='relu')(out)
    out = tf.keras.layers.Conv2D(1024, (3, 3), padding='same', activation='relu')(out)
    out = ASPPBlock()(out)
    out = tf.keras.layers.Conv2D(512, (3, 3), padding='same', activation='relu')(out)

    out = CTConCC(512, (3, 3), (2, 2), padding='same', activation='relu')(out, out4)    
    out = CTConCC(256, (3, 3), (2, 2), padding='same', activation='relu')(out, out3)    
    out = CTConCC(128, (3, 3), (2, 2), padding='same', activation='relu')(out, out2)    
    out = CTConCC(64,  (3, 3), (2, 2), padding='same', activation='relu')(out, out1)
    
    out = tf.keras.layers.Conv2D(1, (3, 3), padding='same', activation='sigmoid')(out)
    return tf.keras.Model(inputs=x, outputs=out)

model = build_model()

In [17]:
tf.keras.utils.plot_model(model, show_shapes=True)

('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')


In [18]:
NUM_EPOCHS = 1
BATCH_SIZE = 32
INP_SIZE = 128

In [19]:
train = COCO_dataset_train.train_dataset(BATCH_SIZE, NUM_EPOCHS, INP_SIZE)
val = COCO_dataset_val.val_dataset(BATCH_SIZE, INP_SIZE)

In [33]:
val

<BatchDataset shapes: ((32, None, None, None), (32, None, None, None)), types: (tf.float32, tf.float32)>

In [ ]:
model = tf.keras.models.load_model('/tmp/model')

In [ ]:
loss = tf.keras.losses.BinaryCrossentropy()
model.compile(optimizer='adam', loss=loss)
hist = model.fit(train, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS)
plt.plot(hist.history['loss'])

 499/2003 [======>.......................] - ETA: 26:07:26 - loss: 0.4768

In [ ]:
model.save('/tmp/model')

In [ ]:
out = model.predict(img[None, ...])[0, ...]
_=plt.imshow(out[...,0])

In [ ]:
for sample in train_x:
    out = model.predict(sample[None, ...])
    seg_map = (out[0, ..., 0]>0.5).astype(np.float32)
    seg_map_clr = plt.get_cmap('viridis')(seg_map)[..., :3]
    plt.imshow(sample*0.5 + seg_map_clr*0.5)
    plt.show()